In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from datetime import datetime

# Configuración inicial
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha del análisis: {datetime.now().strftime('%Y-%m-%d %H:%M')}")